## 1. Introduction
The dataset includes the comments of 10 wireless workout earphones, and this analysis report presents a comparison between different products by sentiment analysis. 

## 2. Objectives
In order to figure out the pros and cons between different products. Exploring the customer comments is an idea to do compatitor analysis. It also helps make a customer segmentation before issuing a new product. This report chooses 3 products, boAt Rockerz 255, Skullcandy S2PGHW-174, and JBL T110BT, in the same price range, under $30. Here comes the quetions to continue an analysis.

1. What concerns customers in terms of product features?
2. Sentiment analysis and the predicsion?
3. The relationshiop between the number of words used for comments and the review stars?

## 3. Method
Python programing

## 4. Prepare
* First of all, import required functions and detemine the location of the dataset to support the analysis.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import seaborn as sns

* Read the dataset; review the elements inside the data; then, choose the data required for the questions to continue the analysis.

In [ ]:
earphone=pd.read_csv('../input/amazonearphonesreviews/AllProductReviews.csv')

In [ ]:
earphone.head()

* Explore how many different products in the Product column of this dataset.

In [ ]:
unique_list=[]
unique=[unique_list.append(x) for x in earphone['Product'] if x not in unique_list]
print(unique_list)

* After reviewing, boAt Rockerz 255, Skullcandy S2PGHW-174, and JBL T110BT are the targets in the same price range, $30. Thus, here is extracting the data and separately save them into different dataframe.

In [ ]:
boat=earphone[earphone['Product']=="boAt Rockerz 255"]
skullcandy=earphone[earphone['Product']=="Skullcandy S2PGHW-174"]
jbl=earphone[earphone['Product']=="JBL T110BT"]

## 5. Process
* Look into the data in order to make sure whether there is a null value.

In [ ]:
print(boat.isna().sum())
print(skullcandy.isna().sum())
print(jbl.isna().sum())

## 6. Analyze
* Combine all review comments together into a long string in order to generate the input for WordCloud.

In [ ]:
boat_review=boat['ReviewBody'].reset_index()
skullcandy_review=skullcandy['ReviewBody'].reset_index()
jbl_review=jbl['ReviewBody'].reset_index()

In [ ]:
boat_str=boat_review['ReviewBody'].str.cat(sep=' ')
skullcandy_str=skullcandy_review['ReviewBody'].str.cat(sep=' ')
jbl_str=jbl_review['ReviewBody'].str.cat(sep=' ')

* Plot these 3 products by WordCloud to investigate what concerns customers the most. The bigger words, the more important to investigate the preferences and pain points of the customers. Here is firstly to use the defult stopwords to filter out the commonly unnecessary words, such as 'I', 'it', and 'the,' for only the boAt earphone in order to figure out whether other stopwords should be added.

In [ ]:
boat_wc=WordCloud(background_color='white', width=1080, height=720, stopwords=STOPWORDS).generate(boat_str)
plt.imshow(boat_wc, interpolation='bilinear')
plt.axis('off')
plt.title('boAt Rockerz 255',size=16, y=1.04)
plt.show()

* After reviewing the first plot, here is adding more stopwords, such as the brand name, product types, and verbs, to improve the exhibit. Also, since the objective is to understand the preferred product features, elimiating exclamation such as good, nice, and better to let product features stand out. However, here still keeps negative words in case there are many conplains as the pain points.

In [ ]:
stopwords_re=set(STOPWORDS)
stopwords_re.update(['product', 'earphone', 'earphones', 'use', 'boat', 'jbl', 'skullcandy', 'got', 'using', 'ear', 'phone', 'headphone', 'headphones', 'good', 'nice', 'great', 'sound', 'quality', 'one', 'month', 'buy', 'amazon'])
boat_wc=WordCloud(background_color='white', width=1080, height=720, stopwords=stopwords_re).generate(boat_str)
plt.imshow(boat_wc, interpolation='bilinear')
plt.axis('off')
plt.title('boAt Rockerz 255',size=16, y=1.04)
plt.show()

In [ ]:
skullcandy_wc=WordCloud(background_color='#e0edfd', width=1080, height=720, stopwords=stopwords_re).generate(skullcandy_str)
plt.imshow(skullcandy_wc, interpolation='bilinear')
plt.axis('off')
plt.title('Skullcandy S2PGHW-174',size=16, y=1.04)
plt.show()

In [ ]:
jbl_wc=WordCloud(background_color='#000000', width=1080, height=720, stopwords=stopwords_re).generate(jbl_str)
plt.imshow(jbl_wc, interpolation='bilinear')
plt.axis('off')
plt.title('JBL T110BT',size=16, y=1.04)
plt.show()

The sentiment analysis starts from here. The extra stopwords are also applied into the ENGLISH_STOP_WORDS list.

In [ ]:
english_stopwords_re=ENGLISH_STOP_WORDS.union(['product', 'earphone', 'earphones', 'use', 'boat', 'jbl', 'skullcandy', 'got', 'using', 'ear', 'phone', 'headphone', 'headphones', 'good', 'nice', 'great', 'sound', 'quality', 'one', 'month', 'buy', 'amazon'])
boat_vector=TfidfVectorizer(stop_words=english_stopwords_re, ngram_range=(1, 2), max_features=200,token_pattern=r'\b[^\d\W][^\d\W]+\b').fit(boat_review['ReviewBody'])
x_b=boat_vector.transform(boat_review['ReviewBody'])
boat_transformed = pd.DataFrame(x_b.toarray(), columns=boat_vector.get_feature_names())
boat_transformed.head()

In [ ]:
skullcandy_vector=TfidfVectorizer(stop_words=english_stopwords_re, ngram_range=(1, 2), max_features=200, token_pattern=r'\b[^\d\W][^\d\W]+\b').fit(skullcandy_review['ReviewBody'])
x_s=skullcandy_vector.transform(skullcandy_review['ReviewBody'])
skullcandy_transformed = pd.DataFrame(x_s.toarray(), columns=skullcandy_vector.get_feature_names())
skullcandy_transformed.head()

In [ ]:
jbl_vector=TfidfVectorizer(stop_words=english_stopwords_re, ngram_range=(1, 2), max_features=200, token_pattern=r'\b[^\d\W][^\d\W]+\b').fit(jbl_review['ReviewBody'])
x_j=jbl_vector.transform(jbl_review['ReviewBody'])
jbl_transformed = pd.DataFrame(x_j.toarray(), columns=jbl_vector.get_feature_names())
jbl_transformed.head()

* Here starts determining the polarity and the sentiment score of these 3 products in order to test and predict by using this dataset. Test accuracy and classification report to understand how proper the outcome is.

In [ ]:
boat_score=boat['ReviewBody']
boat_score.head()

In [ ]:
list_b1=[]
for i in range(len(boat_score)):
    sentiment_b1=TextBlob(boat_score.iloc[i])
    sentiment_b1_score=sentiment_b1.sentiment.polarity
    list_b1.append(sentiment_b1_score)
boat['polarity']=pd.DataFrame({'ReviewBody':list_b1})

In [ ]:
list_b2=[]
for i in range(len(boat_score)):
    sentiment_b2=TextBlob(boat_score.iloc[i])
    sentiment_b2_score=sentiment_b2.sentiment.polarity
    if sentiment_b2_score > 0:
        binary_score=1
    else:
        binary_score=0
    list_b2.append(binary_score)
boat['score']=pd.DataFrame({'ReviewBody':list_b2})

In [ ]:
boat.head()

In [ ]:
x_train_b, x_test_b, y_train_b, y_test_b = train_test_split(boat_transformed, boat['score'], test_size=0.2, random_state=444)
logistic_regression = LogisticRegression().fit(x_train_b, y_train_b)
y_predicted_b = logistic_regression.predict(x_test_b)

print('Test Accuracy (boAt) -', accuracy_score(y_test_b, y_predicted_b))
print(confusion_matrix(y_test_b, y_predicted_b)/len(y_test_b))
print("Test Classification Report (boAt)\n", classification_report(y_test_b, y_predicted_b))

In [ ]:
skullcandy_score=skullcandy['ReviewBody']
list_s1=[]
for i in range(len(skullcandy_score)):
    sentiment_s1=TextBlob(skullcandy_score.iloc[i])
    sentiment_s1_score=sentiment_s1.sentiment.polarity
    list_s1.append(sentiment_s1_score)
skullcandy['polarity']=list_s1
list_s2=[]
for i in range(len(skullcandy_score)):
    sentiment_s2=TextBlob(skullcandy_score.iloc[i])
    sentiment_s2_score=sentiment_s2.sentiment.polarity
    if sentiment_s2_score > 0:
        binary_score=1
    else:
        binary_score=0
    list_s2.append(binary_score)
skullcandy['score']=list_s2

In [ ]:
skullcandy.head()

In [ ]:
x_train_s, x_test_s, y_train_s, y_test_s = train_test_split(skullcandy_transformed, skullcandy['score'], test_size=0.2, random_state=444)
logistic_regression = LogisticRegression().fit(x_train_s, y_train_s)
y_predicted_s = logistic_regression.predict(x_test_s)

print('Test Accuracy (Skullcandy) -', accuracy_score(y_test_s, y_predicted_s))
print(confusion_matrix(y_test_s, y_predicted_s)/len(y_test_s))
print("Test Classification Report (Skullcandy)\n", classification_report(y_test_s, y_predicted_s))

In [ ]:
jbl_score=jbl['ReviewBody']
list_j1=[]
for i in range(len(jbl_score)):
    sentiment_j1=TextBlob(jbl_score.iloc[i])
    sentiment_j1_score=sentiment_j1.sentiment.polarity
    list_j1.append(sentiment_j1_score)
jbl['polarity']=list_j1
list_j2=[]
for i in range(len(jbl_score)):
    sentiment_j2=TextBlob(jbl_score.iloc[i])
    sentiment_j2_score=sentiment_j2.sentiment.polarity
    if sentiment_j2_score > 0:
        binary_score=1
    else:
        binary_score=0
    list_j2.append(binary_score)
jbl['score']=list_j2

In [ ]:
jbl.head()

In [ ]:
x_train_j, x_test_j, y_train_j, y_test_j = train_test_split(jbl_transformed, jbl['score'], test_size=0.2, random_state=444)
logistic_regression = LogisticRegression().fit(x_train_j, y_train_j)
y_predicted_j = logistic_regression.predict(x_test_j)

print('Test Accuracy (JBL) -', accuracy_score(y_test_j, y_predicted_j))
print(confusion_matrix(y_test_j, y_predicted_j)/len(y_test_j))
print("Test Classification Report (JBL)\n", classification_report(y_test_j, y_predicted_j))

In [ ]:
boat_word_tokens=[word_tokenize(review) for review in boat['ReviewBody']]
boat_tokens=[]
for i in range(len(boat_word_tokens)):
    boat_tokens.append(len(boat_word_tokens[i]))
boat['tokens']=boat_tokens

In [ ]:
boat.head()

* Lastly, here is to calcualte the average number of words that is used for the comments and then to prepare charts to demonstrate the stars for these three products. Also, violin charts are to show the whether there is a pattern of words used regarding different stars.

In [ ]:
skullcandy_word_tokens=[word_tokenize(review) for review in skullcandy['ReviewBody']]
skullcandy_tokens=[]
for i in range(len(skullcandy_word_tokens)):
    skullcandy_tokens.append(len(skullcandy_word_tokens[i]))
skullcandy['tokens']=skullcandy_tokens

In [ ]:
skullcandy.head()

In [ ]:
jbl_word_tokens=[word_tokenize(review) for review in jbl['ReviewBody']]
jbl_tokens=[]
for i in range(len(jbl_word_tokens)):
    jbl_tokens.append(len(jbl_word_tokens[i]))
jbl['tokens']=jbl_tokens

In [ ]:
jbl.head()

In [ ]:
sns.set_style('ticks')
sns.catplot(x='ReviewStar', data=boat, kind='count')
sns.set_context("notebook")
plt.title('boAt Rockerz 255 Review',size=16, y=1.04)
plt.xlabel('Review Star')
plt.ylabel('Count Numbers')
plt.show()

In [ ]:
sns.catplot(x='ReviewStar', data=skullcandy, kind='count')
sns.set_context("notebook")
plt.title('Skullcandy S2PGHW-174 Review',size=16, y=1.04)
plt.xlabel('Review Star')
plt.ylabel('Count Numbers')
plt.show()

In [ ]:
sns.catplot(x='ReviewStar', data=jbl, kind='count')
sns.set_context("notebook")
plt.title('JBL T110BT Review',size=16, y=1.04)
plt.xlabel('Review Star')
plt.ylabel('Count Numbers')
plt.show()

In [ ]:
print('Average Review Words: \nboAt Rockerz 255 -', boat['tokens'].mean(), '\nSkullcandy S2PGHW-174 -', skullcandy['tokens'].mean(), '\nJBL T110BT -', jbl['tokens'].mean())

In [ ]:
sns.violinplot(x='ReviewStar', y='tokens', data=boat)
sns.set_context("notebook")
plt.title('boAt Rockerz 255 - Word Count v.s. Review Stars',size=16, y=1.04)
plt.xlabel('Review Star')
plt.ylabel('Count Numbers')
plt.show()

In [ ]:
sns.violinplot(x='ReviewStar', y='tokens', data=skullcandy)
sns.set_context("notebook")
plt.title('Skullcandy S2PGHW-174 - Word Count v.s. Review Stars',size=16, y=1.04)
plt.xlabel('Review Star')
plt.ylabel('Count Numbers')
plt.show()

In [ ]:
sns.violinplot(x='ReviewStar', y='tokens', data=jbl, kind='count')
sns.set_context("notebook")
plt.title('JBL T110BT - Word Count v.s. Review Stars',size=16, y=1.04)
plt.xlabel('Review Star')
plt.ylabel('Count Numbers')
plt.show()

## 7. Conclusion

After the analysis, here are viewpoints as below.

1. The product features mostly concern customers are the battery life, bass, and price. As for the JBL earphone, it is clear that customers are facing problems that the product does not work by looking in to the graphic.

2. The sentiment analysis provieds good enough results and prediction with a decent accuracy. 

3. From the bar charts, JBL gets a high percentage of 1 star, a negative review, than the other two products. From the violin charts, it is a patern that customers would like to type more words for 1 star or 5 star review.

Suggested further analysis.

1. This analysis uses TextBlob to determine whether the review is positive or negative. However, it should be more accurate to determine the polarity by the stars that customers give.

2. This sentiment analysis demonstrats the test accuracy. A sentiment analysis for the train accuracy can be the next.

3. Since there is a pattern showing 1 star and 5 star reviews get more words. Is there a correlation to represent this situation?